<a href="https://colab.research.google.com/github/OldGoose/Datamining/blob/master/Santa_2021_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab資料夾設定方法
#!pip install google.colab #如未安裝取消註解後執行
import os

from google.colab import drive
drive.mount('/content/drive')

##出現提示欄進行授權

os.chdir('/content/drive/My Drive/Santa2021') #切換該目錄
os.listdir() #確認目錄內容

MessageError: ignored

In [ ]:
import itertools
import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

#!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz
!tar xvfz LKH-3.0.7.tgz
!cd LKH-3.0.7; make

tar (child): LKH-3.0.7.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
/bin/bash: line 0: cd: LKH-3.0.7: No such file or directory
make: *** No targets specified and no makefile found.  Stop.


### 1. 列出 n = 7 的最佳排列
由Bogdan Coanda 所提出的公式 $𝑳(𝒏)=𝒏!+(𝒏−𝟏)!+(𝒏−𝟐)!+(𝒏−𝟑)!+𝒏−𝟑!-1$ On 27 February 2019，  
我們可以計算出$n=7$最短超排列組合是5906

In [ ]:
best = "12345671234561723456127345612374561327456137245613742561374526137456213745612347561324756134275613472561347526134756213475612345761234516723451627345162374516234751623457162345176234512673451263745126347512634571263451726345127634512367451236475123645712364517236451273645123764512346751234657123465172346512734651243765124367512436571243651724365127436512473651246375124635712463517246351274635124763512467351426735146273514672351467325146735216473521674352167345216374521634752163457216345271634521764352176453271645327614532764153276451326745132647513264571326451732645137264531726453712645372164537261453726415372645132764531276453217645231764521376452173654217365241736521473652174365217346521736452176345216735421637542163574216354721635427163542176354216735241637524163572416352741635247163524176352416735214673512465371246531724653127465312476531246753142675314627531467253146752316475321647531264753162475316427531647253164752316745321674531267453162745316724531674253167452316754231675243167523416752314675321467531246573124651372465132746513247651324671532467135246713254671235467125346712543671524367154236715432675143267541326754312675432167543261745362174536127453617245361742536174523617453261743526174325617432651742365174263517426531742651374265173426157342617534216753421765342175634217536421753462175342617354261734526173425617342651743261574362157436125743162574312657413265741236574126357412653741265734126574312567413256741235674125367412563741256734125674312576413257614325761342576132457613254761325746132576412357641253761425376124537612543761524376154237615432761543726154376215437612534761253746125376412573641257634125764312574631257436152743615724361574236157432617543621754361275436172543617524361754236175432671543627154367215436712546371254673125476312547361524736154273615472361547326145736214576321475632147653214763521476325147632154763214576231457621345762143576214537621457361245736142573614527361457236145732614753621475361247536142753614725361475236147532614735261473256147326514723651472635147265314726513472651437265147326154736215473612547316254731265471326547123654712635471265347126543716253471625374162537146253716425371624537162543716524371654237165432716543721654371265473125647132564712356471253647125634712564372156437251643275614327564132756431275643217564327156432751643257163425176342516734251637425163472516342751634257163245176324516732451637245163274516324751632457163254716325741632571463275146327154632714563271465327146352714632571643527164357216435712643517264351276435126743512647351264375126435716243517624351672435162743516247351624375162435716423517642351674235164723516427351642375146237514263751423675142376514273651427635142765314276513427651432765142375614235761423567143256714352671435627143567214356712435617243561274356124735612437561243576124356714235617423561472356142735614237516423571643251764325167432516473251643725614372564137256431725643712564731254671324567132465713246751324615732461753246173524617325416723541762354716235476123547621354762315467231546273154623715462317564231576421356742135647213564271356421735624137562413576241356724135627413562471356241735621473562174356217345621735462173564213756421357642153746215374261537421653742156374215367421537642157364215763421576432157642315674231564723156427315642371564231756243157624315672431562743156247315624371562431756234157623415672341562734156237415623471562341756231475623174562317546321745632174653217463521746325174632157463217546312754631725463175246315724631527463152476315246731524637152463175426315742631547263154276315426731542637154263175462315746235174623571462357416235746123574621357462315476235147623541726354172365417235641723546172354167253417625314762531746253176425317624531762543176524317654231765432176543127654317265431762534172653417256341725364172534617253416725431672541367251436725134672153476215347261534721653472156347215364721534672135467213456721346572136457213654721365742136572413657214365721346752136475213674521367542136752413675214376521437562143752614375216437521463725146372154637214563721465372146357214637521436752134672513647251367425136724513672541637254167325417632541736251473625174362517346257136425713624571362547136257413625714362571346275136427513624751362745136275416327541623754126375412367541237654132765413726541376251437625134762513746251376425137624513762541376524137654213765412375641237546132754613725461375246137542613754621375461237541627354126735412763541273654127356412735461273541627534126753412765341275634127536412753461275341627543162754136275143627513462715342671354267134526713425671342657143265714236571426357142653714265731426571342675134267153427615342716534271563427153642715346271354627134562713465271364527136542713652471365274136527143652713462573146257341625734612573462157346251736425173624517362541732654173256417325461732456173246517324615372461532746153247615324167532416573214657321645731264573162457316425731645273165427316524731652743165273416527314652731645723165472316574231657243165723416572314657231645732165473216574321657342165732416537241653274165324716532417653241567321456731245637124563172456312745631247563124576312456731425637142563174256314725631427563142576314256731452637145236714532671453627145367214536712453671425367145237614523716452371465237416523746152347651234765213476523147652341765234716523476152346715234617523461572346152734615237465123746521374652317465237145623714526317452631475263145726314527631452673145627314567231456732154673215647321567432156734215673241563724156327415632471563241756324157632415367241536274153624715362417536241573624153762415326741532647153264175326415732641523764152367415236471523641752364157236415273641526374152634715263417526341572634152763415267341526437152643175264315726431527643152674315264731526413752641357261435726134572613547261357426135724613572641352761435276134527613542761352476135274613527641352674135264713526417352641"
len(best)

5906

### 2. 將字串分為三個組別
由這個公式$𝑝_𝑘^𝑛 = \frac{𝑛!}{(𝑛−𝑘)!}$ 我們可以得到以下這七種符號(🎅🤶🦌🧝🎄🎁🎀) 共有5040個排列組合(i.e., 共有5040部電影)  
因題目的限制，由這🎅🤶兩個符號開頭的電影共有120部電影，我們必須將由放入每一個組別中。  
扣除那120部電影，我們會將剩餘的4920電影平均分為三個部分。

* 決定🎅🤶要設定哪一個代號

In [ ]:
split_id = [len(best) // 3, len(best)*2//3]
t1, t2 = split_id

strings = [best[:t1], best[t1:t2], best[t2:]]
g1 = strings[0]
g2 = strings[1]
g3 = strings[2]

check = []
for i in range(1,8):
    for j in range(1,8):
        if i != j:
            check.append(str(i)+str(j))
check = np.array(check)

data = pd.DataFrame()
column = list(check)
index = ['group1','group2','group3']
data = pd.DataFrame(columns=column,index=index)

for i in list(check):
    data[i][0] = (g1.count(i))
    data[i][1] = (g2.count(i))
    data[i][2] = (g3.count(i))
data = data.append(data.std(axis= 0) ,ignore_index=True)

print("The Min Standard Deviation is: " + str(data.stack().min()) + " at 24.")

The Min Standard Deviation is: 0.5773502691896258 at 24.


*    找尋沒有🎅🤶開頭的電影

In [ ]:
#用於比對字符是否在單位組合中
def is_perm(s):
    y = True
    for k in range(1,8):
        y = y&(str(k) in s)
        if not y: break
    return y

permutations = []

for k in range(len(best)-6):
    s = best[k:k+7]
    if (is_perm(s))&(s not in permutations)&(s[:2]!='24'):
        permutations.append(s)
        
print('這裡共有',len(permutations),'部電影，要分成三等份')

這裡共有 4920 部電影，要分成三等份


*   找出限制的120部電影

In [ ]:
mandatory = ['24'+''.join(x) for x in itertools.permutations(['1','3','5','6','7'], 5)]
print('另外',len(mandatory),'部由🎅🤶開頭的電影，必須出現在每一個組別中')

另外 120 部由🎅🤶開頭的電影，必須出現在每一個組別中


*   將4920個電影分為三組

In [ ]:
#找出切點
split_id = [len(permutations) // 3, len(permutations)*2//3]
t1, t2 = split_id

#用於微調整這三組的數量
adjust1 = 1
adjust2 = 9
group1 = permutations[:t1-adjust1] + mandatory
group2 = permutations[t1-adjust1:t2+adjust2] + mandatory
group3 = permutations[t2+adjust2:] + mandatory

len(group1), len(group2), len(group3)

(1759, 1770, 1751)

In [ ]:
group1

['1234567',
 '2345671',
 '3456712',
 '4567123',
 '5671234',
 '6712345',
 '7123456',
 '2345617',
 '3456172',
 '4561723',
 '5617234',
 '6172345',
 '1723456',
 '7234561',
 '3456127',
 '4561273',
 '5612734',
 '6127345',
 '1273456',
 '2734561',
 '7345612',
 '4561237',
 '5612374',
 '6123745',
 '1237456',
 '2374561',
 '7456132',
 '4561327',
 '5613274',
 '6132745',
 '1327456',
 '3274561',
 '2745613',
 '4561372',
 '5613724',
 '6137245',
 '1372456',
 '3724561',
 '7245613',
 '5613742',
 '6137425',
 '1374256',
 '3742561',
 '7425613',
 '4256137',
 '2561374',
 '6137452',
 '1374526',
 '3745261',
 '7452613',
 '4526137',
 '5261374',
 '2613745',
 '1374562',
 '3745621',
 '7456213',
 '4562137',
 '5621374',
 '6213745',
 '2137456',
 '3745612',
 '7456123',
 '5612347',
 '6123475',
 '1234756',
 '2347561',
 '4756132',
 '7561324',
 '5613247',
 '6132475',
 '1324756',
 '3247561',
 '7561342',
 '5613427',
 '6134275',
 '1342756',
 '3427561',
 '4275613',
 '2756134',
 '5613472',
 '6134725',
 '1347256',
 '3472561',
 '47

Help function

In [ ]:
def hamming_distance(str1, str2):
    return sum( (c1!=c2) for c1, c2 in zip(str1, str2))

def offset(s1, s2):
    assert(len(s1)==len(s2))
    ln = len(s1)
    j = ln
    for k in range(0,ln):
        if hamming_distance(s1[k:],s2[:7-k])==0:
            j=k
            break
    return j

In [ ]:
def hamming_distance(str1, str2):
    return sum( (c1!=c2) for c1, c2 in zip(str1, str2))

def offset(s1, s2):
    assert(len(s1)==len(s2))
    ln = len(s1)
    j = ln
    for k in range(0,ln):
        if hamming_distance(s1[k:],s2[:7-k])==0:
            j=k
            break
    return j


def get_tsp_solution(group):
    
    # CREATE DISTANCE MATRIX
    SIZE = len(group)
    M = np.zeros((SIZE, SIZE), dtype='int8')
    for j in range(SIZE):
        #if j%25==0: print(j,', ',end='')
        for k in range(SIZE):
            M[j,k] = offset(group[j],group[k])
            
    # WRITE PROBLEM FILE
    f = open(f'group.par','w')
    f.write("PROBLEM_FILE = ../distances.atsp\n")
    f.write("TOUR_FILE = ../output.txt\n")
    f.write(f"OPTIMUM = {SIZE}\n")
    f.write("MOVE_TYPE = 5\n")
    f.write("PATCHING_C = 3\n")
    f.write("PATCHING_A = 2\n")
    f.write("RUNS = 20\n")
    f.write("TIME_LIMIT = 1200\n") #seconds
    f.close()
    
    # WRITE PARAMETER FILE
    f = open(f'distances.atsp','w')
    f.write("NAME: distances\n")
    f.write("TYPE: ATSP\n")
    f.write("COMMENT: Asymmetric CTSP\n")
    f.write(f"DIMENSION: {SIZE}\n")
    f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
    f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
    f.write("EDGE_WEIGHT_SECTION\n")
    for j in range(SIZE):
        #if j%25==0: print(j,', ',end='')
        for k in range(SIZE):
            f.write(f"{M[j,k]:2d} ") 
        f.write("\n")
    f.close()
    
    # EXECUTE TSP SOLVER
    !cd LKH-3.0.7; ./LKH ../group.par
    
    # READ RESULTING ORDER
    with open('output.txt') as f:
        lines = f.readlines()
    for i,ln in enumerate(lines):
        if 'TOUR_SECTION' in ln: break
    perms = [int(x[:-1]) for x in lines[i+1:-2] ]
    
    # CREATE STRING
    result = group[ perms[0]-1 ]
    for k in range(1,len(perms)):
        s1 = group[ perms[k-1]-1 ]
        s2 = group[ perms[k]-1 ]
        d = offset(s1,s2)
        assert(d!=0)
        result += s2[-d:]
    
    return result

In [ ]:
def print_header(x):
    print(); print()
    print('#'*25)
    print('### Computing String',x)
    print('#'*25); print()

print_header(1)
string1 = get_tsp_solution(group1)
print_header(2)
string2 = get_tsp_solution(group2)
print_header(3)
string3 = get_tsp_solution(group3)

### 3. 確認每一部電影及指定電影都有在這三個字串內

In [ ]:
all_permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]

for p in all_permutations:
    if p not in group1 and p not in group2 and p not in group3:
        print(p)

In [ ]:
for p in mandatory:
    if p not in string1:
        print(p)
    if p not in string2:
        print(p)
    if p not in string3:
        print(p)

In [ ]:
result_list = [string1, string2, string3]
for i in result_list:
  print(len(i))

### 4. 將元素轉換為表情符號

In [ ]:
# CONVERT NUMBERS TO EMOJIS
replace_dict = {
 '2': '🎅',
 '4': '🤶',
 '8': '🌟',
 '1': '🦌',
 '3': '🧝',
 '5': '🎄',
 '6': '🎁',
 '7': '🎀'}

for k,v in replace_dict.items():
    string1 = string1.replace(k, v)
    string2 = string2.replace(k, v)
    string3 = string3.replace(k, v)

In [ ]:
# WRITE SUBMISSION CSV
sub = pd.DataFrame()
sub['schedule'] = [string1, string2, string3]
sub.to_csv('submission_no_wildcards_' + str(len(string1)) + '_' +str(len(string2)) + '_' + str(len(string3))+ '.csv',index=False)
sub.head()

In [ ]:
LETTERS = {
    1: '🎅',  # father christmas
    2: '🤶',  # mother christmas
    3: '🦌',  # reindeer
    4: '🧝',  # elf
    5: '🎄',  # christmas tree
    6: '🎁',  # gift
    7: '🎀',  # ribbon
    8: '🌟',  # star
}
INV_LETTERS = {v: k for k, v in LETTERS.items()}

solution = pd.read_csv('/content/drive/My Drive/Santa2021/submission_no_wildcards_2526_2520_2508.csv')
strings = [[INV_LETTERS[c] for c in s] for s in solution.schedule]
strings.sort(key=len, reverse=True)
print(f'Strings lengths are {[len(_) for _ in strings]}.')

In [ ]:
improved, old_lens = True, [len(i) for i in strings]
while improved:
    print('='*91)
    # new_strings = [solve_atsp(i) for i in strings_perms]
    new_strings = list(process_map(solve_atsp, strings_perms))
    new_strings.sort(key=len, reverse=True)
    new_lens = [len(_) for _ in new_strings]
    if new_lens < old_lens:
        print(f'Improved strings lengths from {old_lens} to {new_lens}.')
        strings, old_lens = new_strings, new_lens
        strings_perms = find_strings_perms(strings, verbose=True)
        strings_perms = rebalance_perms(strings_perms, verbose=True)
    else:
        improved = False

In [ ]:
def print_header(x):
    print(); print()
    print('#'*25)
    print('### Computing String',x)
    print('#'*25); print()

print_header(1)
string1 = get_tsp_solution(strings_perms[0])
print_header(2)
string2 = get_tsp_solution(strings_perms[1])
print_header(3)
string3 = get_tsp_solution(strings_perms[2])

In [ ]:
all_permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]

for p in all_permutations:
    if p not in string1 and p not in string2 and p not in string3:
        print(p)

In [ ]:
for p in mandatory:
    if p not in string1:
        print(p)
    if p not in string2:
        print(p)
    if p not in string3:
        print(p)

In [ ]:
def find_strings_perms(strings, verbose=False):
    found_perms = []
    perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
    for s in strings:
        found_perms.append([])
        for i in range(len(s)-6):
            p = s[i:i+7]
            if p in perms:
                found_perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  found_perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  found_perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return found_perms

def change(a):
    a = a.replace("🎅", "2")
    a = a.replace('🤶', "4")
    a = a.replace('🦌', "1")
    a = a.replace('🧝', "3")
    a = a.replace('🎄', "5")
    a = a.replace('🎁', "6")
    a = a.replace('🎀', "7")
    return a
def star_optimization(a):
    symbols = "🎅🤶🦌🧝🎄🎁🎀"
    words = a

    schedules = []
    schedules.append(change(a[0]))
    schedules.append(change(a[1]))
    schedules.append(change(a[2]))

    perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
    found_perms = find_strings_perms(schedules)
    balanced_perms = rebalance_perms(found_perms)
    perm2id = {p: i for i, p in enumerate(perms)}
    perms_arr = np.array([list(map(int, p)) for p in perms])

    perms_onehot = np.eye(7)[perms_arr-1, :].transpose(0, 2, 1)
    assert np.allclose(perms_onehot[:,0,:].astype(np.int64), (perms_arr == 1).astype(np.int64))

    left = perms_onehot[perm2id["1234567"]]
    right = perms_onehot[perm2id["5671234"]]
    matches = F.conv2d(
        F.pad(torch.Tensor(left[None, None, :, :]), (7, 7)),
        torch.Tensor(right[None, None, :, :]),
        padding="valid"
    ).numpy().reshape(-1)
    must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
    must_match_right2left = np.array([0, 1, 2, 3, 4, 5, 6, 7, -1, -1, -1, -1, -1, -1, -1])
    cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

    M = F.conv2d(
        F.pad(torch.Tensor(perms_onehot[:, None, :, :]), (7, 7)),
        torch.Tensor(perms_onehot[:, None, :, :]),
        padding="valid"
    ).squeeze().numpy()

    must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
    must_match_left2right_wild = np.array([-1, -1, -1, -1, -1, -1, -1, 6, 5, 4, 3, 2, 1, 0, 0])

    cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

    costMat = np.where(M == must_match_left2right, cost_ifmatch, np.inf).min(axis=-1).astype(np.int8)
    costMatWild = np.minimum(costMat, np.where(M == must_match_left2right_wild, cost_ifmatch, np.inf).min(axis=-1)).astype(np.int8)

    nodes_list = []
    table_list = []

    for i in range(3):
        word = words[i]
        nodes = [perm2id[p] for p in balanced_perms[i]]

        table = np.zeros((len(nodes), 10), np.int64)
        table[0, :] = 7
        for i in range(1, len(nodes)):
            e = costMat[nodes[i-1], nodes[i]]
            ew = costMatWild[nodes[i-1], nodes[i]]
            table[i,0] = table[i-1,0] + e
            table[i,1] = min(table[i-1,1] + e, table[i-1,0] + ew)
            table[i,2] = min(table[i-1,2], table[i-1,1]) + e
            table[i,3] = min(table[i-1,3], table[i-1,2]) + e
            table[i,4] = min(table[i-1,4], table[i-1,3]) + e
            table[i,5] = min(table[i-1,5], table[i-1,4]) + e
            table[i,6] = min(table[i-1,6], table[i-1,5]) + e
            table[i,7] = min(table[i-1,7], table[i-1,6]) + e
            table[i,8] = min(table[i-1,8], table[i-1,7]) + e
            table[i,9] = min(table[i-1,9] + e, table[i-1,8] + ew)
        nodes_list.append(nodes)
        table_list.append(table)

    # backtrack
    new_words = []
    wilds = []
    for nodes, table in zip(nodes_list, table_list):
        ns = [perms[nodes[-1]]]
        track = np.argmin(table[-1])
        wild = []
        for i in range(len(nodes)-2, -1, -1):
            e = costMat[nodes[i], nodes[i+1]]
            ew = costMatWild[nodes[i], nodes[i+1]]
            if track == 0:
                ns.append(perms[nodes[i]][:e])
            elif track == 1:
                if table[i, 1] + e < table[i, 0] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    ns.append(perms[nodes[i]][:ew])
                    track = track - 1
            elif 2 <= track <= 8:
                if table[i, track] >= table[i, track-1]:
                    track = track - 1
                ns.append(perms[nodes[i]][:e])
            elif track == 9:
                if table[i, 9] + e < table[i, 8] + ew:
                    ns.append(perms[nodes[i]][:e])
                else:
                    ns.append(perms[nodes[i]][:ew])
                    left = np.array(list(map(int, perms[nodes[i]][ew:])))
                    right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                    mis = np.where(left != right)[0][0]
                    wild.append(table[i, track-1]-7+ew+mis)
                    track = track - 1
            else:
                assert False
        assert track == 0
        wilds.append(wild)
        nsw = list("".join(ns[::-1]))
        for w in wild:
            nsw[w] = "*"
        new_words.append("".join(nsw))

    print("Optimisation done")
    print("score: ", max(map(len, words)), "->", max(map(len, new_words)))
    sub = [a.translate(str.maketrans("1234567*", symbols+"🌟")) for a in new_words]
    if check_if_good(sub): print("Optimization validated")

    return sub

def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms]
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != "24" and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != "24" and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms]
def check_if_good(a):
    is_false = False
    error_count_start = 0
    error_count_other = 0
    # Check if the submission is valid
    wildcard = pd.read_csv('/content/drive/My Drive/Santa2021/wildcards.csv')
    permus = pd.read_csv('/content/drive/My Drive/Santa2021/permutations.csv')
    start = permus[permus.Permutation.str[:2] =='🎅🤶']
    other = permus[permus.Permutation.str[:2] !='🎅🤶'].reset_index(drop=True)

    # If all the combinations beginning with 🎅🤶 are in the three submissions
    for permu in start['Permutation'].tolist():
        if permu in a[0]:
            if permu in a[1]:
                if permu in a[2]:
                    continue
        # For wildcards
        permus_for_wild = wildcard[wildcard['Permutation']==permu].index.values
        in_string = False
        for p in permus_for_wild:
            if wildcard.at[p, 'Factor'] in a[0]: in_string = True
            if wildcard.at[p, 'Factor'] in a[1]: in_string = True
            if wildcard.at[p, 'Factor'] in a[2]: in_string = True
        if in_string: continue
        is_false = True
        #print("Not all 🎅🤶, missing:", permu)
        error_count_start += 1

    # If all the combinations are in the submissions
    for permu in other['Permutation'].tolist():
        if permu in a[0]: continue
        if permu in a[1]: continue
        if permu in a[2]: continue

        # For wildcards
        permus_for_wild = wildcard[wildcard['Permutation']==permu].index.values
        in_string = False
        for p in permus_for_wild:
            if wildcard.at[p, 'Factor'] in a[0]: in_string = True
            if wildcard.at[p, 'Factor'] in a[1]: in_string = True
            if wildcard.at[p, 'Factor'] in a[2]: in_string = True
        if not in_string:
            is_false = True
            #print("Not all the combinaison, missing:", permu)
            error_count_other += 1

    # If there are 2 stars or less
    if a[0].count('🌟') > 2:
        print("Too many stars in string: 0")
        return False
    if a[1].count('🌟') > 2:
        print("Too many stars in string: 1")
        return False
    if a[2].count('🌟') > 2:
        print("Too many stars in string: 2")
        return False

    if is_false:
        print("Nombre d'erreur start:", error_count_start, ', other', error_count_other)
        return False
    return True

In [ ]:
paths = star_optimization(result_list)